In [113]:
pip install vecstack

In [0]:
#Import all libraries
from vecstack import stacking
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

In [115]:
 #To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [116]:
#Read training data file

trainfile = r'/gdrive/My Drive/508 - A6/Churn-Train-1.csv'
trainData = pd.read_csv(trainfile)

#Read test data file
testfile = r'/gdrive/My Drive/508 - A6/Churn-Test-1.csv'
testData = pd.read_csv(testfile)

# copy of train data and test data to perform pre processing
#cpy_traindata = trainData.copy()
#cpy_testdata = testData.copy()

trainData.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [117]:
#Shape and data types of data
print("Training Data shape:",trainData.shape)
print("Test Data shape:",testData.shape)
print("Training Data types:",trainData.dtypes)
# variable data types
trainData.describe()

Training Data shape: (1165, 21)
Test Data shape: (1262, 21)
Training Data types: State              object
Account Length      int64
Area Code           int64
Phone              object
Int'l Plan         object
VMail Plan         object
VMail Message       int64
Day Mins          float64
Day Calls           int64
Day Charge        float64
Eve Mins          float64
Eve Calls           int64
Eve Charge        float64
Night Mins        float64
Night Calls         int64
Night Charge      float64
Intl Mins         float64
Intl Calls          int64
Intl Charge       float64
CustServ Calls      int64
TARGET              int64
dtype: object


,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
count,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000,1165.000000
mean,100.107296,437.899571,8.319313,181.588412,100.605150,30.870532,199.351073,99.871245,16.945021,199.931159,100.104721,8.996927,10.228155,4.550215,2.762172,1.539056,0.125322
std,40.128933,42.861654,13.730382,54.392831,19.819381,9.246717,52.633766,20.077314,4.473876,51.233628,19.899559,2.305449,2.833913,2.538757,0.765126,1.307352,0.331226
min,1.000000,408.000000,0.000000,12.500000,30.000000,2.130000,31.200000,12.000000,2.650000,43.700000,42.000000,1.970000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,73.000000,408.000000,0.000000,146.800000,88.000000,24.960000,164.400000,87.000000,13.970000,167.000000,87.000000,7.520000,8.400000,3.000000,2.270000,1.000000,0.000000
50%,98.000000,415.000000,0.000000,182.300000,101.000000,30.990000,200.200000,100.000000,17.020000,200.500000,101.000000,9.020000,10.300000,4.000000,2.780000,1.000000,0.000000
75%,128.000000,510.000000,21.000000,216.000000,114.000000,36.720000,234.100000,114.000000,19.900000,236.600000,114.000000,10.650000,12.100000,6.000000,3.270000,2.000000,0.000000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,351.600000,168.000000,29.890000,364.300000,175.000000,16.390000,20.000000,19.000000,5.400000,9.000000,1.000000


In [118]:
# percentage missing data per column in training data
trainData_na = (trainData.isnull().sum() / len(trainData)) * 100
trainData_na = trainData_na.drop(trainData_na[trainData_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :trainData_na})
missing_data['No_of_rows_missing'] = trainData.isnull().sum()
print("Missing data in train\n",missing_data)
# percentage missing data per column in test data
testData_na = (testData.isnull().sum() / len(testData)) * 100
testData_na = testData_na.drop(testData_na[testData_na == 0].index).sort_values(ascending=False)
missing_data_test = pd.DataFrame({'Missing Ratio' :testData_na})
missing_data_test['No_of_rows_missing'] = testData.isnull().sum()
print("Missing data in test\n",missing_data_test)

Missing data in train
                 Missing Ratio  No_of_rows_missing
State                     NaN                   0
Account Length            NaN                   0
Area Code                 NaN                   0
Phone                     NaN                   0
Int'l Plan                NaN                   0
VMail Plan                NaN                   0
VMail Message             NaN                   0
Day Mins                  NaN                   0
Day Calls                 NaN                   0
Day Charge                NaN                   0
Eve Mins                  NaN                   0
Eve Calls                 NaN                   0
Eve Charge                NaN                   0
Night Mins                NaN                   0
Night Calls               NaN                   0
Night Charge              NaN                   0
Intl Mins                 NaN                   0
Intl Calls                NaN                   0
Intl Charge               N

In [119]:
#remove 1 row with data missing from test data
testData.dropna(subset=(['TARGET','CustServ Calls',"Intl Charge","Intl Calls","Intl Mins","Night Charge", "Night Calls"]), how='all', inplace=True)
print("After removing row with missing values-", testData.shape)


After removing row with missing values- (1261, 21)


In [0]:
#Copy Train data excluding target
X_train = trainData.iloc[:, :-1].copy()

#Select just Target Column
y_train = trainData.iloc[:, -1]

In [0]:
#Copy Test data excluding target
X_test = testData.iloc[:, :-1].copy()

#Select just Target Column
y_test = testData.iloc[:, -1]

In [122]:
testData.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,TARGET
0,VT,50,415,387-5891,yes,yes,26,307.1,94,52.21,289.4,78,24.60,174.9,109.0,7.87,8.0,3.0,2.16,0.0,0.0
1,UT,72,415,368-8026,no,no,0,118.2,106,20.09,167.2,136,14.21,214.2,106.0,9.64,12.2,3.0,3.29,3.0,0.0
2,KS,130,510,332-9446,no,no,0,154.0,95,26.18,205.9,106,17.50,233.7,75.0,10.52,12.9,1.0,3.48,1.0,0.0
3,NV,143,408,393-5284,no,no,0,155.5,101,26.44,213.4,89,18.14,237.9,61.0,10.71,7.6,11.0,2.05,1.0,0.0
4,DE,89,510,376-1677,yes,no,0,125.6,108,21.35,213.0,90,18.11,181.7,108.0,8.18,5.4,5.0,1.46,1.0,0.0


In [0]:
#List of Categorical Features
categoricalFeatures = ["State", "Area Code", "Int'l Plan", "VMail Plan"]

In [124]:
#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([X_train, X_test], keys=[0,1])

#Do one Hot encoding for categorical features for combined data
combined_data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

combined_data = combined_data.drop(["Phone"], axis=1)

#Separate Train data and test data
X_Train = combined_data.xs(0)
X_Test = combined_data.xs(1)

X_Train = pd.DataFrame(X_Train)
X_Test = pd.DataFrame(X_Test)

print("Test data",X_Test.shape)
print("Training data", X_Train.shape)

Test data (1261, 73)
Training data (1165, 73)


In [125]:
X_Test.head()

,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,State_AK,State_AL,State_AR,State_AZ,State_CA,State_CO,State_CT,State_DC,State_DE,State_FL,State_GA,State_HI,State_IA,State_ID,State_IL,State_IN,State_KS,State_KY,State_LA,State_MA,State_MD,State_ME,State_MI,State_MN,State_MO,State_MS,State_MT,State_NC,State_ND,State_NE,State_NH,State_NJ,State_NM,State_NV,State_NY,State_OH,State_OK,State_OR,State_PA,State_RI,State_SC,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area Code_408,Area Code_415,Area Code_510,Int'l Plan_no,Int'l Plan_yes,VMail Plan_no,VMail Plan_yes
0,50,26,307.1,94,52.21,289.4,78,24.60,174.9,109.0,7.87,8.0,3.0,2.16,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1
1,72,0,118.2,106,20.09,167.2,136,14.21,214.2,106.0,9.64,12.2,3.0,3.29,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0
2,130,0,154.0,95,26.18,205.9,106,17.50,233.7,75.0,10.52,12.9,1.0,3.48,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
3,143,0,155.5,101,26.44,213.4,89,18.14,237.9,61.0,10.71,7.6,11.0,2.05,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
4,89,0,125.6,108,21.35,213.0,90,18.11,181.7,108.0,8.18,5.4,5.0,1.46,1.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0


In [126]:
# check if there are any missing values in any column
print("Number of null values in train data",X_Train.isnull().sum().sum())
print("Number of null values in test data",X_Test.isnull().sum().sum())
# no missing values

Number of null values in train data 0
Number of null values in test data 0


In [127]:
#Decision Tree Classifier ========================================================================
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_Train, y_train)
model_score = clf.score(X_Train, y_train)
print('coefficient of determination R^2 of the prediction.:',model_score)
clf_predict_Train=clf.predict(X_Test)

#clf.feature_importances_
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, clf_predict_Train)))


print('Test Variance score: %.2f' % r2_score(y_test, clf_predict_Train))

coefficient of determination R^2 of the prediction.: 1.0
Root Mean squared error: 0.29
Test Variance score: 0.37


In [128]:
#Print Classification Report
print(classification_report(y_test,clf_predict_Train))

              precision    recall  f1-score   support

         0.0       0.94      0.96      0.95      1069
         1.0       0.77      0.67      0.71       192

    accuracy                           0.92      1261
   macro avg       0.85      0.82      0.83      1261
weighted avg       0.91      0.92      0.92      1261



In [129]:
roc_auc_score(y_test,clf_predict_Train)

0.8150919862800123

In [130]:
#MLP ============================================================
model =MLPClassifier()
model = model.fit(X_Train, y_train)
MLP_score = model.score(X_Train, y_train)
print('coefficient of determination R^2 of the prediction.:',MLP_score)


y_pred = model.predict(X_Test)

# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, y_pred))

coefficient of determination R^2 of the prediction.: 0.8909871244635194
Root Mean squared error: 0.39
Test Variance score: -0.16


In [131]:
#Print Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92      1069
         1.0       0.62      0.04      0.08       192

    accuracy                           0.85      1261
   macro avg       0.73      0.52      0.50      1261
weighted avg       0.82      0.85      0.79      1261



In [132]:
roc_auc_score(y_test, y_pred)

0.5184946990957282

In [133]:
#Random Forest Classifier==============================================================================

rfr = RandomForestClassifier()
rfr.fit(X_Train, y_train)
RF_score = rfr.score(X_Train, y_train)
print('coefficient of determination R^2 of the prediction.:',RF_score)
rfr_predict_Train=rfr.predict(X_Test)

#clf.feature_importances_
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, rfr_predict_Train)))


print('Test Variance score: %.2f' % r2_score(y_test, rfr_predict_Train))

coefficient of determination R^2 of the prediction.: 0.9948497854077253
Root Mean squared error: 0.32
Test Variance score: 0.22


In [134]:
#Print Classification Report
print(classification_report(y_test, rfr_predict_Train))

              precision    recall  f1-score   support

         0.0       0.90      1.00      0.94      1069
         1.0       0.95      0.36      0.52       192

    accuracy                           0.90      1261
   macro avg       0.92      0.68      0.73      1261
weighted avg       0.90      0.90      0.88      1261



In [135]:
roc_auc_score(y_test, rfr_predict_Train)

0.6778165926099159

In [136]:
#SVM ============================================================
from sklearn.svm import SVC
model =SVC()
model = model.fit(X_Train, y_train)
SVC_score = model.score(X_Train, y_train)
print('coefficient of determination R^2 of the prediction.:',SVC_score)

y_predictedsvm = model.predict(X_Test)

# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, y_predictedsvm)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, y_predictedsvm))


coefficient of determination R^2 of the prediction.: 1.0
Root Mean squared error: 0.39
Test Variance score: -0.18


In [137]:
#Print Classification Report
print(classification_report(y_test, y_predictedsvm))

              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92      1069
         1.0       0.00      0.00      0.00       192

    accuracy                           0.85      1261
   macro avg       0.42      0.50      0.46      1261
weighted avg       0.72      0.85      0.78      1261



In [138]:
roc_auc_score(y_test, y_predictedsvm)

0.5

In [144]:
# Hyper parameter Tuning Decision Tree Classifier
clf = DecisionTreeClassifier()
parameters={'criterion':('gini','entropy'),'min_samples_split' : range(10,50,10),'max_depth': range(1,10,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=5)
clf_random.fit(X_Train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_Train,y_train)


{'min_samples_split': 30, 'max_depth': 7, 'criterion': 'gini'}


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [145]:
clf_predict = clf.predict(X_Test)
# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, clf_predict)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, clf_predict))

Root Mean squared error: 0.27
Test Variance score: 0.44


In [146]:
#Print Classification Report
print(classification_report(y_test, clf_predict))

              precision    recall  f1-score   support

         0.0       0.93      0.99      0.96      1069
         1.0       0.94      0.56      0.70       192

    accuracy                           0.93      1261
   macro avg       0.93      0.78      0.83      1261
weighted avg       0.93      0.93      0.92      1261



In [147]:
roc_auc_score(y_test, clf_predict)

0.7779759120673527

In [151]:
# Hyper parameter Tuning MLP Classifier
clf1 =MLPClassifier()
parameters={'activation':('identity', 'logistic'),'solver' : ('lbfgs', 'sgd'),'learning_rate': ('constant', 'invscaling')}
clf1_random = RandomizedSearchCV(clf1,parameters,n_iter=5)
clf1_random.fit(X_Train, y_train)
grid_parm=clf1_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf1 = MLPClassifier(**grid_parm)
clf1.fit(X_Train,y_train)


{'solver': 'lbfgs', 'learning_rate': 'invscaling', 'activation': 'identity'}


MLPClassifier(activation='identity', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='invscaling',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [152]:
clf1_predict = clf1.predict(X_Test)
# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, clf1_predict)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, clf1_predict))

Root Mean squared error: 0.39
Test Variance score: -0.18


In [153]:
#Print Classification Report
print(classification_report(y_test, clf1_predict))

              precision    recall  f1-score   support

         0.0       0.85      1.00      0.92      1069
         1.0       0.00      0.00      0.00       192

    accuracy                           0.85      1261
   macro avg       0.42      0.50      0.46      1261
weighted avg       0.72      0.85      0.78      1261



In [154]:
roc_auc_score(y_test, clf1_predict)

0.5

In [155]:
# Hyper parameter Tuning Random Forest Classifier
clf2 = RandomForestClassifier()
parameters = {'n_estimators': range(50,100,10),
              'min_samples_split' : range(10,100,10),
              'max_depth': range(1,20,2)
              }

clf2_random = RandomizedSearchCV(clf2,parameters,n_iter=5)
clf2_random.fit(X_Train, y_train)
grid_parm=clf2_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf2 = RandomForestClassifier(**grid_parm)
clf2.fit(X_Train,y_train)


{'n_estimators': 60, 'min_samples_split': 40, 'max_depth': 19}


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=19, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=40,
                       min_weight_fraction_leaf=0.0, n_estimators=60,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [156]:
clf2_predict = clf2.predict(X_Test)
# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, clf2_predict)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, clf2_predict))

Root Mean squared error: 0.35
Test Variance score: 0.06


In [157]:
#Print Classification Report
print(classification_report(y_test, clf2_predict))

              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93      1069
         1.0       0.98      0.21      0.34       192

    accuracy                           0.88      1261
   macro avg       0.93      0.60      0.64      1261
weighted avg       0.89      0.88      0.84      1261



In [158]:
roc_auc_score(y_test, clf2_predict)

0.6036989398191456

In [167]:
# Hyper parameter Tuning SVM Classifier
from sklearn.svm import LinearSVC
clf3 = LinearSVC()
parameters = {'multi_class': ('crammer_singer', 'ovr'), 'max_iter' : range(1,1000,50)
              }

clf3_random = RandomizedSearchCV(clf3,parameters,n_iter=5)
clf3_random.fit(X_Train, y_train)
grid_parm=clf3_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf3 = LinearSVC(**grid_parm)
clf3.fit(X_Train,y_train)

{'multi_class': 'ovr', 'max_iter': 151}


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=151,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [168]:
clf3_predict = clf3.predict(X_Test)
# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, clf3_predict)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, clf3_predict))

Root Mean squared error: 0.39
Test Variance score: -0.18


In [169]:
#Print Classification Report
print(classification_report(y_test, clf2_predict))

              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93      1069
         1.0       0.98      0.21      0.34       192

    accuracy                           0.88      1261
   macro avg       0.93      0.60      0.64      1261
weighted avg       0.89      0.88      0.84      1261



In [170]:
roc_auc_score(y_test, clf2_predict)

0.6036989398191456

In [173]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_Train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))



___________________________________________________________________
SMOTE

Resampled dataset shape Counter({0: 1019, 1: 509})


In [174]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ MLPClassifier(solver= 'lbfgs', learning_rate= 'invscaling', activation= 'identity'), 
          RandomForestClassifier(n_estimators= 60, min_samples_split= 40, max_depth= 19), DecisionTreeClassifier(min_samples_split= 30, max_depth= 7, criterion= 'gini') ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_Test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=3, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [MLPClassifier]
    fold  0:  [0.66666667]
    fold  1:  [0.66666667]
    fold  2:  [0.66535433]
    ----
    MEAN:     [0.66622922] + [0.00061864]
    FULL:     [0.66623037]

model  1:     [RandomForestClassifier]
    fold  0:  [0.92352941]
    fold  1:  [0.91176471]
    fold  2:  [0.91929134]
    ----
    MEAN:     [0.91819515] + [0.00486507]
    FULL:     [0.91819372]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90588235]
    fold  1:  [0.89411765]
    fold  2:  [0.90551181]
    ----
    MEAN:     [0.90183727] + [0.00546069]
    FULL:     [0.90183246]



In [176]:
#STACKING - CONTRUCT A Random Forest MODEL==============================
model = RandomForestClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
# The mean squared error
print("Root Mean squared error: %.2f"% np.sqrt(mean_squared_error(y_test, y_pred)))
# Explained variance score: 1 is perfect prediction
print('Test Variance score: %.2f' % r2_score(y_test, y_pred))

Root Mean squared error: 0.31
Test Variance score: 0.26


In [177]:
#Print Classification Report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.97      0.95      1069
         1.0       0.75      0.56      0.64       192

    accuracy                           0.90      1261
   macro avg       0.84      0.76      0.79      1261
weighted avg       0.90      0.90      0.90      1261



In [178]:
roc_auc_score(y_test, y_pred)

0.7622753936700967